# NLP & Classification Group Project

### Project Title: News Article Category Predictions
#### Done By: Amogelang Mogwane, Chris Phillip, Moosa Molibeli & Tiisetso Gabaza

© ExploreAI 2024

----

![NewsPaper](newspaper1.jpg)

<a id="cont"></a>
## Table of Contents

<a href=#one>1. Background Context</a>

<a href=#two>2. Importing Packages</a>

<a href=#three>3. Data Collection and Description</a>

<a href=#four>4. Loading Data </a>

<a href=#five>5. Data Cleaning and PreProcessing</a>

<a href=#six>6. Exploratory Data Analysis (EDA)</a>

<a href=#seven>7. Modeling</a>

<a href=#eight>8. Evaluation</a>

<a href=#nine>9. Final Model</a>

<a href=#ten>10. Streamlit Deployment</a>

<a href=#eleven>11. Conclusion and Future Work</a>

<a href=#twelve>12. References</a>

---
<a id="one"></a>
## Background Context
<a href=#cont>Back to Table of Contents</a>

From the inception of printed newspapers, every article has been assigned to a specific section. While many aspects of the newspaper industry have evolved—ranging from ink and paper types to distribution methods—the practice of categorizing news has persisted across generations, extending into digital formats. Newspaper articles cover a broad spectrum of topics, including politics, sports, and entertainment. Historically, categorization was primarily a manual process, but advancements in technology now allow for automated classification with minimal effort.

This project aims to design and develop an application that predicts the categories of news articles intended for publication. By utilizing classification algorithms, we will analyze the content of articles to determine their respective genres. The proposed algorithm will not only classify existing topics but also adapt to new topics as they emerge in the world. While the algorithm is extendable to multiple languages, this paper will primarily focus on English.

In the realm of news article classification, multi-label text classification poses a significant challenge. Our objective is to assign one or more category labels to each article. For each category, classifiers will provide binary responses—either "yes" or "no"—indicating whether a specific category applies to the given test data. This approach utilizes binary classifiers, and we will implement several standard algorithms, including Naive Bayes, Support Vector Machines, and Logistic Regression, which are commonly used for binary classification. We will evaluate these approaches and select the best model based on predetermined parameters for predicting news categories.

A robust category classification algorithm for news articles must achieve high acuuracy while remaining easily updatable. Given the continuous evolution of news topics and events, the ability to seamlessly add new categories to the classifier is essential.

![News Articles](newspaper2.jpg)

---
<a id="two"></a>
## Importing Packages
<a href=#cont>Back to Table of Contents</a>

**Please Note:**
*The below are all the libraries I believe I will need for this project. This list will be adjusted as needed throughout the project*

In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns
import nltk
import re
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.combine import SMOTEENN 
from sklearn.model_selection import train_test_split

from wordcloud import WordCloud

from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV

import streamlit as st

import joblib 

import warnings
warnings.filterwarnings('ignore')

In [5]:
# please run this code if you do not have wordcloud installed in your environment
#!pip install wordcloud

In [6]:
# please run this code if you do not have imbalanced-learn installed in your environment
#pip install imbalanced-learn

In [7]:
# please run this code if you do not have xgboost installed in your environment
#!pip install xgboost

In [8]:
# please run this code if you do not have xgboost installed in your environment
#!pip install streamlit

---
<a id="three"></a>
## Data Collection and Description
<a href=#cont>Back to Table of Contents</a>

This project utilizes a news article dataset containing articles categorized into five distinct groups: Sports, Business, Entertainment, Education, and Technology. The dataset has already been randomly partitioned into training and testing sets, with the challenge of developing a classification model to predict the category of each news article.

The training dataset comprises 5,520 records across five columns, while the testing dataset consists of 2,000 observations with the same column structure. The columns included in the dataset are: 'index', 'headlines', 'description', 'content', 'url', and 'category'. The target variable for prediction is the 'category' column. Other columns will either be discarded (e.g., the 'url' column) or combined to create a single content column for the articles.

Below is a description of the dataset features:

**Dataset Features:**
| **Column**                                                                                  | **Description**              
|---------------------------------------------------------------------------------------------|--------------------   
| Headlines   | 	The headline or title of the news article.
| Description | A brief summary or description of the news article.
| Content | The full text content of the news article.
| URL | The URL link to the original source of the news article.
| Category | The category or topic of the news article (e.g., business, education, entertainment, sports, technology).

---
<a id="four"></a>
## Loading Data
<a href=#cont>Back to Table of Contents</a>

As the datasets have been provided in csv formats, I will upload both of the train.csv and test.csv datasets. The approach taken in this workbook is that we will follow the same steps to preprocess both datasets in order to maintain alignment when validation of our model is required in the evaluation step of this project. 

In [9]:
# loading our training and our testing data sets
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [10]:
# getting a sense of the training data from the first 5 observations
train_df.head()

,headlines,description,content,url,category
0,RBI revises definition of politically-exposed ...,The central bank has also asked chairpersons a...,The Reserve Bank of India (RBI) has changed th...,https://indianexpress.com/article/business/ban...,business
1,NDTV Q2 net profit falls 57.4% to Rs 5.55 cror...,NDTV's consolidated revenue from operations wa...,Broadcaster New Delhi Television Ltd on Monday...,https://indianexpress.com/article/business/com...,business
2,"Akasa Air ‘well capitalised’, can grow much fa...",The initial share sale will be open for public...,Homegrown server maker Netweb Technologies Ind...,https://indianexpress.com/article/business/mar...,business
3,India’s current account deficit declines sharp...,The current account deficit (CAD) was 3.8 per ...,India’s current account deficit declined sharp...,https://indianexpress.com/article/business/eco...,business
4,"States borrowing cost soars to 7.68%, highest ...",The prices shot up reflecting the overall high...,States have been forced to pay through their n...,https://indianexpress.com/article/business/eco...,business


Above we printed the first five rows of the dataset to ensure that it has loaded correctly into our notebook. 

Below we will check the shape of both datasets to ensure that they do have 5,520 and 2,000 observation as we anticipate. 

In [11]:
# understanding the structure of the training dataset
train_df.shape

(5520, 5)

In [12]:
# understanding the structure of the testing dataset
test_df.shape

(2000, 5)

Observation:

From observation, I can indeed confirm that both datasets have loaded correctly and completely into the notebook.

---
<a id="five"></a>
## Data Cleaning and PreProcessing
<a href=#cont>Back to Table of Contents</a>

Data cleaning is an essential first step in any data-driven project, ensuring the dataset is accurate, consistent, and ready for analysis. For our news article classification project, this involves several processes to improve data quality.

We will start by creating a copy of the training dataset. This way, any major changes can be made while preserving the original for reference.
All the steps we do to our training dataset, we will also apply to our testing dataset

In [13]:
# creating a copy of the training dataframe
train_df_copy = train_df.copy()
test_df_copy = test_df.copy()

Next, we'll address any missing or incomplete data. Missing values can distort results and lead to inaccurate conclusions, so it's crucial to identify and handle these gaps through imputation, removal, or replacement.

In [14]:
# checking for missing values
train_df_copy.isnull().sum()

headlines      0
description    0
content        0
url            0
category       0
dtype: int64

In [15]:
# checking for missing values
test_df_copy.isnull().sum()

headlines      0
description    0
content        0
url            0
category       0
dtype: int64

Since there are no missing values, we can confidently move on to the next phase of the project.

Next, we will address any duplicate records to avoid bias in model training. Ensuring each article is unique is essential for accurately training the classification model.

In [16]:
# dropping any duplicates that might exist
train_df_copy.drop_duplicates(inplace=True)

train_df_copy.shape

(5520, 5)

In [17]:
# dropping any duplicates that might exist
test_df_copy.drop_duplicates(inplace=True)

test_df_copy.shape

(2000, 5)

The structure of the training dataset remains intact, confirming there are no duplicates. We can proceed with confidence, knowing our dataset is unbiased for modeling.

Next, we will remove the URL column, as it does not add value to our news category predictions and is considered redundant.

In [18]:
# removing redundant url column 
train_df_copy.drop(columns="url",inplace=True)
test_df_copy.drop(columns="url",inplace=True)

train_df_copy.head()

,headlines,description,content,category
0,RBI revises definition of politically-exposed ...,The central bank has also asked chairpersons a...,The Reserve Bank of India (RBI) has changed th...,business
1,NDTV Q2 net profit falls 57.4% to Rs 5.55 cror...,NDTV's consolidated revenue from operations wa...,Broadcaster New Delhi Television Ltd on Monday...,business
2,"Akasa Air ‘well capitalised’, can grow much fa...",The initial share sale will be open for public...,Homegrown server maker Netweb Technologies Ind...,business
3,India’s current account deficit declines sharp...,The current account deficit (CAD) was 3.8 per ...,India’s current account deficit declined sharp...,business
4,"States borrowing cost soars to 7.68%, highest ...",The prices shot up reflecting the overall high...,States have been forced to pay through their n...,business


Lastly, we will standardize the text data by converting the text in the three feature columns to lowercase for consistency, removing punctuation, eliminating stop words that add little meaning, and lemmatizing the dataset. These steps will streamline the data, making it easier for classification algorithms to process effectively. Lemmatization will reduce words to their base forms, enhancing classification across all observations.

We will define a function to perform these final cleanup steps on dataframes. Recall that what we do to the training dataset will also be applied to the testing dataset.

In [19]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# text cleaning 
def clean_text(text):
    
    text = re.sub(r'\W', ' ', text)  # removing special characters
    text = text.lower()  # lowercasing the text
    tokens = word_tokenize(text)  # tokenizing the text
    
    # removing stop words and lemmatizing to reduce the words to base words
    lemmatizer = WordNetLemmatizer()  # initialization
    stop_words = set(stopwords.words('english'))
    cleaned_tokens = [
        lemmatizer.lemmatize(token) for token in tokens if token not in stop_words
    ]

    # joining tokens back to string
    return ' '.join(cleaned_tokens)

columns_to_clean = ['headlines', 'description', 'content']

# applying the clean_text function to the relevant columns in the train and test datasets
train_df_copy[columns_to_clean] = train_df_copy[columns_to_clean].applymap(clean_text)
test_df_copy[columns_to_clean] = test_df_copy[columns_to_clean].applymap(clean_text)

In [20]:
# printing the firs five observations to check if the datacleaning function worked
train_df_copy.head()

,headlines,description,content,category
0,rbi revise definition politically exposed pers...,central bank also asked chairperson chief exec...,reserve bank india rbi changed definition poli...,business
1,ndtv q2 net profit fall 57 4 r 5 55 crore impa...,ndtv consolidated revenue operation r 95 55 cr...,broadcaster new delhi television ltd monday re...,business
2,akasa air well capitalised grow much faster ce...,initial share sale open public subscription ju...,homegrown server maker netweb technology india...,business
3,india current account deficit decline sharply ...,current account deficit cad 3 8 per cent gdp u...,india current account deficit declined sharply...,business
4,state borrowing cost soar 7 68 highest far fiscal,price shot reflecting overall higher risk aver...,state forced pay nose weekly auction debt tues...,business


By applying these data cleaning techniques, we aim to build a high-quality dataset that improves the reliability of our classification model, ultimately leading to more accurate predictions of news categories.

---
<a id="six"></a>
## Exploratory Data Analysis (EDA)
<a href=#cont>Back to Table of Contents</a>

Exploratory Data Analysis (EDA) is a crucial step in our news classification project, allowing us to better understand the dataset's characteristics and distributions. In this phase, we will focus firstly on visualizing the distribution of news categories to identify any imbalances. This initial visualization will provide insights into how the categories are spread across the dataset.

In [21]:
# getting the distribution of the categories
plt.figure(figsize=(10, 5))
sns.countplot(data=train_df_copy, x='category')
plt.title('Distribution of News Categories\n')
plt.show()

From the visualisation, we can clearly note some imbalances in the spread of our news categories. This can lead to a biased models that perform well on majority classes but poorly on minority classes. A balanced dataset generally helps the model generalize better, leading to improved performance in real-world applications where class distributions may vary.

Let us now numerically verify the counts for each category. Understanding the extent of this imbalance is essential for implementing effective solutions.

In [22]:
# verifying the count of each category
train_df_copy['category'].value_counts()

category
education        1520
technology       1280
business         1120
entertainment     960
sports            640
Name: count, dtype: int64

The above further verifies the imbalance of the categories in our training dataset. 

The correct approach would be to  use Synthetic Data Generation which uses techniques like SMOTE (Synthetic Minority Over-sampling Technique) to create synthetic examples for minority classes. However, after running a SMOTE techinque. we noted that it negatively affected the performance of the models. As a result, we have elected to leave the class imblanace as is, and use the F1-score for evaluation metrics. The F1-score is a better measure as it gives a balance between precision and recall, moreso in the wake of uneven class distribution.
It is important that we vectorize our data. Vectorizing the data serves to convert unstructured text into structured numerical data that machine learning models can process. It also captures the significance of words and their relationships in the context of different categories using methods like TF-IDF or Bag-of-Words. This project will use the TD-IDF method.

Once vectorised, we will then split our training data using the 80-20 split. For this project, we will train and validate the top 3 models on the train.csv data. And then select the best performing model at the end to validate on the test.csv data. This will ensure that that model can be tested on unseen data to evaluate its performance.ique

In [23]:
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.combine import SMOTEENN 
from sklearn.model_selection import train_test_split

# preparing our features and target from the training and testing datasets
X_train = train_df_copy[['headlines', 'description', 'content']]  # feature train column
y_train = train_df_copy['category']  # target train column
X_test = test_df_copy[['headlines', 'description', 'content']]  # feature column for testing
y_test = test_df_copy['category']  # target column for testing

# splitting the training dataset into train and test data using a 20/80 split
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train,  # Features
    y_train,  # Target
    test_size=0.2,  # 20% for validation, 80% for training
    random_state=42,  # For reproducibility
    stratify=train_df_copy['category']  # Ensures the split maintains the class distribution
)

# vectorising the training data
vectorizer = TfidfVectorizer() 
# fitting and transforming the training data
X_train_vec = vectorizer.fit_transform(X_train_split['headlines'] + ' ' + X_train_split['description'] + ' ' + X_train_split['content'])

# transforming the validation data 
X_val_vec = vectorizer.transform(X_val_split['headlines'] + ' ' + X_val_split['description'] + ' ' + X_val_split['content'])

# applying SMOTE to the training data
#smote = SMOTE(random_state=42)
#X_resampled, y_resampled = smote.fit_resample(X_train_vec, y_train)

#combining SMOTE with random sampling on training data
#smote_enn = SMOTEENN(random_state=42)
#X_resampled, y_resampled = smote_enn.fit_resample(X_train_vec, y_train)

# Check the new class distribution
#print(pd.Series(y_resampled).value_counts())

We will now store the vectorised model. This will ensure consistency when applying our Streamlit app. When we vectorised our data, we created a specific mapping of text features. Saving this model will ensure we continue to use the same instance in our Streamlit app when we apply it to our project. 

In [24]:
# saving vectoriser to be used for streamlit app later
import joblib  # For loading the model
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']

Furthermore, we will create word clouds for each category to visualize the most frequently occurring terms. By defining a function to generate these word clouds, we can gain additional insights into the key topics and themes present within each category.

In [25]:
from wordcloud import WordCloud

def generate_word_clouds(df, category_column='category', text_columns=['headlines', 'description', 'content']):
    categories = df[category_column].unique()  # Getting unique categories

    # Ensuring there are no more than 5 categories
    if len(categories) > 5:
        raise ValueError("This function supports up to 5 categories only.")
    
    plt.figure(figsize=(15, 10))  # Setting figure size
    
    for i, category in enumerate(categories):
        plt.subplot(3, 2, i + 1)  # Creating a subplot for each category (adjust grid size)
        
        # Combine text from all specified columns for the current category
        text = ' '.join(df[df[category_column] == category][text_columns].apply(lambda x: ' '.join(x), axis=1))
        
        # Generate the word cloud for the combined text
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
        
        # Display the word cloud
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')  # Turn off axis
        plt.title(category)  # Set the title to the category

    plt.tight_layout()  # Adjust subplots to fit into the figure area
    plt.show()

In [26]:
generate_word_clouds(train_df_copy)

The EDA phase has provided a comprehensive overview of our dataset, helping us identify class imbalances and the key characteristics of each news category. By visualizing category distributions, verifying counts, and generating word clouds, we have laid a solid foundation for subsequent modeling efforts, ultimately enhancing the performance of our classification model.

----
<a id="seven"></a>
## Modeling
<a href=#cont>Back to Table of Contents</a>

The modeling stage is a critical phase in a data science project where the goal is to build a machine learning model (or several models) that can make accurate predictions or insights based on the data.

Given that the category column is a categorical variable, it is important that we label- encode it. Label encoding is a technique used in machine learning to convert categorical labels (text values) into numerical values.

In [27]:
from sklearn.preprocessing import LabelEncoder

# Instantiate the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the target labels to numeric values
y_train_split_encoded = label_encoder.fit_transform(y_train_split)

Training multiple classification models helps us selecting the best performing model, improving generalization, handling different types of data, and increasing robustness. So next, we will train five classification models and evaluate them using cross-validation method. Cross-validation is a technique used to evaluate the performance of a machine learning model by splitting the dataset into multiple subsets (or folds) and training/testing the model on different combinations of these subsets. This will help improve the generalizability of a model, help in detecting overfitting, and provide a more accurate estimate of model performance.

In [28]:
# training multiple classification models

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# instantiating classifiers
models = {
    'Logistic Regression': LogisticRegression(class_weight='balanced'),
    'Naive Bayes': MultinomialNB(),
    'SVM': SVC(class_weight='balanced'),
    'Random Forest': RandomForestClassifier(class_weight='balanced'),
    'XGBoost': xgb.XGBClassifier(class_weight='balanced')
}

# Train models and store them
model_results = {}
for name, model in models.items():
    model.fit(X_train_vec, y_train_split_encoded)
    model_results[name] = model

In [29]:
# model evaluation using cross-validation
from sklearn.model_selection import cross_val_score

# Perform cross-validation on each model
cv_results = {}
for name, model in model_results.items():
    cv_scores = cross_val_score(model, X_train_vec, y_train_split_encoded, cv=5, scoring='accuracy')
    cv_results[name] = cv_scores.mean()

# Display cross-validation results
for name, score in cv_results.items():
    print(f'{name}: Cross-validation Accuracy: {score:.4f}')


Logistic Regression: Cross-validation Accuracy: 0.9789
Naive Bayes: Cross-validation Accuracy: 0.9644
SVM: Cross-validation Accuracy: 0.9769
Random Forest: Cross-validation Accuracy: 0.9601
XGBoost: Cross-validation Accuracy: 0.9633


Observation:

From the above results, we note that Logistic Regression (0.9789) has the highest cross-validation accuracy among all the models, suggesting it is the most effective model for this particular problem. The slight edge in performance could be due to the simplicity and efficiency of this model in capturing linear relationships in the data.

SVM (0.9769) and Naive Bayes (0.9644) also perform well but slightly behind logistic regression.The close performance between SVM and Logistic Regression suggests that the data might be relatively simple, as both models are linear classifiers (though SVM can use kernels for non-linear data).
Naive Bayes, though slightly less accurate, still performs well. It may indicate that feature independence assumptions (used in Naive Bayes) hold reasonably true for the data.

For the next step, we will fine tune the performance of these top 3 performing models in order to enhance their performance. We will use the GridSearch approach to do this. Grid Search Cross-Validation (GridSearchCV) is a technique used for hyperparameter tuning in machine learning models. The purpose is to systematically explore a specified set of hyperparameters for a given model to find the combination that yields the best performance. This will help us in ensuring that we get the best possible configuration for our model. In our news article classification project, this leads to better accuracy, model reliability, and generalization, which are essential for creating a robust and effective classifier.

In [30]:
# hyperparameter tuning using GridSearchCV

from sklearn.model_selection import GridSearchCV

param_grids = {
    'Logistic Regression': {
        'C': [0.1, 1, 10]  # Regularization strength
    },
    'SVM': {
        'C': [0.1, 1, 10],  # Regularization parameter
        'kernel': ['linear', 'rbf'],  # Linear or Radial basis function kernel
        'gamma': [1, 0.1, 0.01, 0.001],  # Kernel coefficient for 'rbf' (if kernel='rbf')
    },
    'Naive Bayes': {
        'alpha': [0.1, 0.5, 1.0, 2.0]  # Smoothing parameter (Laplace smoothing)
    }
}

# Perform GridSearchCV for each model
best_models = {}
for name, model in model_results.items():
    if name in param_grids:
        grid_search = GridSearchCV(model, param_grids[name], cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train_vec, y_train_split_encoded)
        best_models[name] = grid_search.best_estimator_
        print(f"Best Parameters for {name}: {grid_search.best_params_}")

Best Parameters for Logistic Regression: {'C': 10}
Best Parameters for Naive Bayes: {'alpha': 0.1}
Best Parameters for SVM: {'C': 10, 'gamma': 1, 'kernel': 'linear'}


Observation: 

Both Logistic Regression and SVM have C = 10, indicating that the optimal models for this dataset benefit from less regularization and are more flexible in fitting the data. Naive Bayes is optimal with a small alpha = 0.1, indicating that the features likely have strong predictive power, and the model does not need heavy adjustments for unseen data. This likely implies that the dataset has clean and informative features. The best parameters across these models indicate that regularization, feature separability, and model flexibility are key factors in achieving optimal performance for this news article classification task.

---
<a id="eight"></a>
## Evaluation 
<a href=#cont>Back to Table of Contents</a>

The evaluation and validation stage of a data science project is crucial for assessing the performance and reliability of the model. It involves using various metrics to ensure that the model generalizes well to unseen data and performs effectively in real-world scenarios. In this phase, we evaluate how well the model predicts outcomes based on a set of predefined criteria of accuracy, F1-scores and weighted avg. 

The approach we have taken in this notebook is to evaluate the three models using the the classification report. 

In [31]:
# Fit and transform the target labels to numeric values
y_val_split_encoded = label_encoder.fit_transform(y_val_split)

In [32]:
# evaluating the best model
from sklearn.metrics import classification_report, accuracy_score

# Evaluate each best model on the test set
for name, model in best_models.items():
    y_pred = model.predict(X_val_vec)
    print(f'Classification Report for {name}:')
    print(classification_report(y_val_split_encoded, y_pred))
    print(f'Accuracy: {accuracy_score(y_val_split_encoded, y_pred):.4f}')


Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       224
           1       1.00      0.99      0.99       304
           2       1.00      1.00      1.00       192
           3       0.98      0.98      0.98       128
           4       0.98      0.98      0.98       256

    accuracy                           0.99      1104
   macro avg       0.99      0.99      0.99      1104
weighted avg       0.99      0.99      0.99      1104

Accuracy: 0.9864
Classification Report for Naive Bayes:
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       224
           1       1.00      1.00      1.00       304
           2       0.99      1.00      0.99       192
           3       0.98      0.97      0.98       128
           4       0.98      0.98      0.98       256

    accuracy                           0.99      1104
   macro avg       0.99     

Observation

**Accuracy Comparison:**

Logistic Regression and Naive Bayes have the highest accuracy (both 0.9864), while SVM has slightly lower accuracy (0.9846). These differences in accuracy are small, but Logistic Regression and Naive Bayes are tied for the best.

**Macro and Weighted Averages:**

Both Logistic Regression and Naive Bayes have higher weighted average f1-scores (0.99 vs 0.98 for SVM), which means they handle class imbalances better and perform consistently across all classes.

**Precision, Recall, and F1-Score for each class:**

Logistic Regression and Naive Bayes perform almost identically across all classes, with very high precision, recall, and f1-scores for each class. While SVM is strong overall, it has slightly lower recall and precision for class 0 (label 'business') and class 4 (label 'technology'). This suggests that SVM might miss or misclassify a few instances of these categories compared to Logistic Regression or Naive Bayes.

Key Insights:
Logistic Regression and Naive Bayes have identical results, both with a slight edge over SVM in terms of handling all classes equally well. Given that these models are tied in performance metrics, either one of them could be considered the best.

Logistic Regression is often preferred when you are dealing with text data in multi-class classification problems. It’s fast and provides well-calibrated probabilities. If interpretability is important, Logistic Regression also tends to provide clearer insights into the importance of each feature.

Both models will provide excellent performance, but Logistic Regression is likely the preferred choice in this case due to its overall balance between accuracy and interpretability.

---
<a id="nine"></a>
## Final Model
<a href=#cont>Back to Table of Contents</a>

Now that we concluded on the Logistic Regression model being the model that provides superior performance for our news article category prediction, we will retrain the model on the full train.csv dataset, and then predict and validate the model performance on unseen data from the train.csv dataset. 

In [33]:
# 1. Preprocess and vectorize the training and test data
vectorizer = TfidfVectorizer()  # Example: Bi-grams and Tri-grams

# Fit and transform the training data
X_train_vec = vectorizer.fit_transform(train_df_copy['headlines'] + ' ' + train_df_copy['description'] + ' ' + train_df_copy['content'])

# Transform the test data using the same vectorizer (do NOT fit it again)
X_test_vec = vectorizer.transform(test_df_copy['headlines'] + ' ' + test_df_copy['description'] + ' ' + test_df_copy['content'])

# 2. Encode the target labels for both training and test sets
label_encoder = LabelEncoder()

# Fit and transform the training target labels
y_train_encoded = label_encoder.fit_transform(train_df_copy['category'])

# Transform the test target labels (use the same encoder, do not fit again)
y_test_encoded = label_encoder.transform(test_df_copy['category'])

# 3. Train Logistic Regression on the entire training dataset
logreg = LogisticRegression(class_weight='balanced')  # Use class_weight='balanced' if you have class imbalance
logreg.fit(X_train_vec, y_train_encoded)

# 4. Make predictions on the test dataset
y_pred = logreg.predict(X_test_vec)

# 5. Optionally, convert the numeric predictions back to original labels
y_pred_original = label_encoder.inverse_transform(y_pred)

# 6. Evaluate model performance on the test set
print(classification_report(y_test_encoded, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97       400
           1       1.00      0.98      0.99       400
           2       0.99      0.99      0.99       400
           3       0.99      0.99      0.99       400
           4       0.94      0.97      0.96       400

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



The model achieved an overall accuracy of 98%, which indicates that it performs exceptionally well across all classes. This means that the model correctly classifies 98% of the total instances in the dataset, suggesting a high level of reliability and robustness in its predictions.

Precision, Recall, and F1-score are consistently high across all classes, ranging from 0.94 to 1.00. This suggests that the model is well-calibrated and performs reliably across all categories, with no class being underrepresented or overrepresented. Specifically:
* Class 1 has the highest precision (1.00), meaning it has perfect accuracy in classifying instances of this category.
* Class 4 has the lowest precision (0.94), but still performs well, indicating some room for improvement in correctly identifying this category.

Recall (sensitivity) is high for all classes, with the lowest value being 0.96 for class 0. This means that the model successfully identifies most of the actual instances of each class.

he model exhibits excellent performance with high accuracy, balanced precision and recall across all classes, and strong F1-scores. However, slight improvement can be made in class 4, where precision is slightly lower (0.94), but the overall model is very robust.

---
<a id="ten"></a>
## Streamlit Deployment
<a href=#cont>Back to Table of Contents</a>

This section of the project focuses on creating an interactive web application that allows users to interact with the trained model in a user-friendly interface. Streamlit is a powerful framework that enables quick and easy development of data science and machine learning apps. By deploying the model with Streamlit, we provide a simple, visual way for users to input new data (such as news articles), view predictions, and explore model results. This section will guide you through the process of converting the trained model into a deployable web app, allowing for real-time predictions and ensuring that the model is accessible and easy to use for both technical and non-technical users.

In [34]:
# saving the model, vectorizer, and label encoder using joblib
joblib.dump(logreg, 'final_logreg_model.pkl')  # Save the trained Logistic Regression model
joblib.dump(vectorizer, 'final_vectorizer.pkl')  # Save the fitted TfidfVectorizer
joblib.dump(label_encoder, 'final_label_encoder.pkl')  # Save the LabelEncoder

print("Model, Vectorizer, and Label Encoder saved successfully!")

Model, Vectorizer, and Label Encoder saved successfully!


In [35]:
# loading the saved model, vectorizer, and label encoder
logreg = joblib.load('final_logreg_model.pkl')
vectorizer = joblib.load('final_vectorizer.pkl')
label_encoder = joblib.load('final_label_encoder.pkl')

# Example text to classify
text = "Example news article content here."

# Transform the text using the same vectorizer
text_vectorized = vectorizer.transform([text])

# Predict the category using the loaded model
y_pred = logreg.predict(text_vectorized)

# Convert the numeric prediction back to the original label
y_pred_original = label_encoder.inverse_transform(y_pred)

print(f"Predicted Category: {y_pred_original[0]}")

Predicted Category: technology


In [36]:
# running the streamlit app code

import streamlit as st

# loading the saved model and vectorizer
model = joblib.load('final_logreg_model.pkl')
vectorizer = joblib.load('final_vectorizer.pkl')

# Function to predict the category
def predict_category(text):
    # Transform the input text using the same vectorizer
    text_vectorized = vectorizer.transform([text])
    
    # Get the prediction from the model
    prediction = model.predict(text_vectorized)
    
    # Return the predicted class label
    return prediction[0]

# Streamlit app layout
st.title('News Classification Application')
st.subheader('Enter a news article to classify its category')

# Input text box
article_text = st.text_area('Enter article text:', height=300)

# When the user clicks the classify button
if st.button('Classify'):
    if article_text.strip() != "":
        category = predict_category(article_text)
        st.write(f'Predicted Category: {category}')
    else:
        st.write("Please enter some text to classify.")

2024-10-20 16:29:07.718 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-20 16:29:07.718 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-20 16:29:07.718 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-20 16:29:07.718 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-20 16:29:07.718 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-20 16:29:07.718 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-20 16:29:07.718 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-20 16:29:07.718 Session state does not function when running a script without `streamlit run`
2024-10-20 16:29

In [37]:
app_code = """
import streamlit as st
import joblib
import numpy as np

# Load the saved model, vectorizer, and label encoder
logreg = joblib.load('final_logreg_model.pkl')
vectorizer = joblib.load('final_vectorizer.pkl')
label_encoder = joblib.load('final_label_encoder.pkl')

# Function to predict the category
def predict_category(text):
    # Transform the input text using the same vectorizer
    text_vectorized = vectorizer.transform([text])
    
    # Get the prediction from the model
    prediction = logreg.predict(text_vectorized)
    
    # Return the predicted class label
    return label_encoder.inverse_transform(prediction)[0]

# Streamlit app layout
st.title('News Classification Application')
st.subheader('Enter a news article to classify its category')

# Input text box
article_text = st.text_area('Enter article text:', height=300)

# When the user clicks the classify button
if st.button('Classify'):
    if article_text.strip() != "":
        category = predict_category(article_text)
        st.write(f'Predicted Category: {category}')
    else:
        st.write("Please enter some text to classify.")
"""

# Save the code to a file named 'app.py'
with open("app.py", "w") as f:
    f.write(app_code)

print("app.py file has been saved successfully!")

app.py file has been saved successfully!


---
<a id="eleven"></a>
## Conclusion and Future Work
<a href=#cont>Back to Table of Contents</a>

The News Article Category Prediction project successfully developed a machine learning model capable of classifying news articles into predefined categories. Through a series of well-defined steps, the project focused on data preprocessing, model selection, and evaluation to ensure the final model provides accurate and reliable predictions. Key stages included cleaning and transforming the dataset, exploring and addressing class imbalance, and leveraging multiple classification algorithms to find the best-performing model. After rigorous testing and hyperparameter tuning using GridSearchCV, Logistic Regression emerged as the most effective model, outperforming others based on accuracy and F1-score metrics.

Additionally, the project culminated in the development of an interactive web application using Streamlit, enabling real-time predictions of news article categories. This makes the model accessible to users, allowing them to input new articles and receive predictions in an intuitive interface.

#### Considerations for Future Work

While the current model provides strong performance, there are several areas for potential improvement and future exploration:

1. Handling Class Imbalance:
Although the project identified a class imbalance, the decision to avoid SMOTE due to performance issues should be revisited. Exploring other techniques such as class weighting or undersampling may yield better results without affecting model performance.

2. Text Feature Engineering:
Further improvements could be made to text feature extraction, such as exploring word embeddings (e.g., Word2Vec, GloVe) or contextual embeddings like BERT for a more nuanced understanding of the text data.

3. Real-Time Model Updates:
As the news domain is dynamic and constantly evolving, regular model updates could be implemented to ensure the classifier remains accurate and relevant. Automating the retraining process with fresh data would help maintain model performance over time.

---
<a id="twelve"></a>
## References
<a href=#cont>Back to Table of Contents</a>

[1] Rao, S., Sudarshan, K. and Abhishek (2020) 'News Article Category Predictor', Department of Computer Science and Engineering, Srinivas Institute of Technology, Valachil, India.

[2] Tong, S. and Koller, D. (2000) 'Support vector machine active learning with applications to text classification', in Langley, P. (ed.) Proceedings ICML-00, 17th International Conference on Machine Learning, pp. 999–1006.

[3] McCallum, A. and Nigam, K. (1998) 'A comparison of event models for naive Bayes text classification', in AAAI/ICML-98 Workshop on Learning for Text Categorization.